In [1]:
import Pkg

Pkg.add("Interpolations")
Pkg.add("PyPlot")
Pkg.add("PyCall")
Pkg.add("OrdinaryDiffEq")
Pkg.add("YAML")
Pkg.add("DelimitedFiles")
Pkg.add("CSV")
Pkg.add("HDF5")
Pkg.add("StructArrays")
Pkg.add("Random")
Pkg.add("Statistics")

using Interpolations, PyCall, OrdinaryDiffEq,
YAML, DelimitedFiles, CSV, HDF5, StructArrays, Random, 
NBInclude, PyPlot, Statistics

ct = pyimport("cantera")

   Updating registry at `~/.julia/registries/General`


  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


...

No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


  Installed Trapz ─ v2.0.3


Updating `~/GCresearch/rocketemissions/Project.toml`
  [592b5752] 

+ Trapz v2.0.3
Updating `~/GCresearch/rocketemissions/Manifest.toml`
  [592b5752] + Trapz v2.0.3


PyObject <module 'cantera' from '/opt/conda/envs/lae2020/lib/python3.8/site-packages/cantera/__init__.py'>

In [5]:
### COMPUTE u AND T AND χ

struct gas_type
    gas
end

### COMPUTE u AND T: FUNCTIONS ###

function compute_y(u::AbstractVector, ρ_mean::AbstractFloat, 
    Δψ::AbstractVector)
    return (2 * cumsum((1/(ρ_mean .* u)) .* Δψ) .^ 0.5)
end

compute_y (generic function with 1 method)

In [6]:
### COMPUTE u AND T ###
# No chemistry

## Constants/Inputs ##
h = [16000] # altitude to caluclate plume properties at [m]
h_string = "16000"
n = 100 # steps in x/ϕ direction
s = 200 # steps in y/ψ direction
m = 1 # counter for test set to only test single altitude
Pr = 1.0 # prandtl number
Le = 1.0 # lewis number
n_species = 53 # number of species in mechanism file

# create spacing arrays for ϕ and ψ
Δϕ = 0.1 * ones(n) #step size in phi
for i = 2:n
    Δϕ[i] = 1.05 * Δϕ[i-1] #enlarge with each step by 1.05
end

Δψ = 0.1 * ones(s) #s vertical grid points
for i = 2:s
    Δψ[i] = 1.05 * Δψ[i-1] #enlarge with each step by 1.05
end

## Initialize loop over altitudes of interest

for m = 1:length(h)
    h_string = string(h[m])

    # Import rocket exit conditions from upstream (u, T, p)
    u0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/u") # initial plume velocity [m/s]
    u0 = convert(AbstractFloat, u0[2])
    T0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T") # initial plume temperature [K]
    T0 = convert(AbstractFloat, T0[2])
    p_all = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/P") #ambient pressure [Pa]
    p = convert(AbstractFloat, p_all[2])
    χ0_full = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/X")
    χ0 = zeros(length(χ0_full[:,2]))

    for d = 1: length(χ0_full[:,2])
        χ0[d] = χ0_full[d,2] # save PPM of species exiting rocket plume
    end

    #curve fit #a = ambient vel [m/s] (speed of rocket) #NEED TO UPDATE W HORIZONTAL VELOCITY
    u_a = 1.11849E-19 * big(h[m])^5 - 1.14814E-14 * big(h[m])^4 + 4.22542E-10 * big(h[m])^3 - 6.92322E-06 * big(h[m])^2 + 6.58761E-02 * big(h[m]) + 5.37920E+01
    T_a = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T_a") # ambient atmospheric temperature [K]
    χ_a = zeros(n_species)
    # set each individual species to respective PPM for ambient air, assume air composition does not change significantly with altitude
    χ_a[48] = 780790 #ppm N2
    χ_a[4] = 209445 #ppm O2
    χ_a[49] = 0 #ppm Ar #CHANGE BACK AFTER DEBUG TO 9339
    χ_a[16] = 9765 #ppm CO2 #426

    #CHECK THAT IMPORTED CONDITIONS ARE CORRECT

    ## Initialize initial u and T conditions: at x/ϕ = 0, y/ψ = all
    gas_init = ct.Solution("gri30.yaml") # create a gas object tuple to define the density with Cantera
    gas_amb = ct.Solution("gri30.yaml") # create a gas object tuple to define the density with Cantera
    gas_g = StructArray{gas_type}(undef,s,n,length(h)) # make tuple to hold gas objects for each ϕ/ψ point at each altitude
    gas_g .= [gas_type(0)] # initialize tuple
    gas_init.TPX = T0, p, χ0 # set state of gas to be rocket exhaust conditions
    print(gas_init)
    #gas_g.gas[:,1,m] .= gas_init[2] # save exhaust conditions in tuple for all ψ and ϕ = 1, and m = altitude index
    gas_amb.TPX = T_a, p, χ_a # set state of gas to be ambient conditions

    u_init = u0 .* ones(s) # array of length y/ψ filled with rocket exit velocity value [m/s]
    T_init = T0 .* ones(s) # array of length y/ψ filled with rocket exit temperature value [K]
    ρ_mean = (gas_init.density_mass[1] + gas_amb.density_mass[1])/2 # NOTE not the best way but cant figure out circular logic btw density and y NOTE
    # we need to define the initial plume geometry by filling the initial u and T arrays with ambient u and T values outside of the plume radius
    radius = 1.147 # rocket exit plume radius [m]

    # have to map ψ in y domain to map properties to the correct grid-points (inside or outside plume radius)
    y_init = compute_y(u_init, ρ_mean, Δψ) #REDERIVE

    # if y is greater than the radius of the plume, assign the ambient conditions to that point
    u_init[y_init.>radius] .= convert(AbstractFloat, u_a)
    T_init[y_init.>radius] .= convert(AbstractFloat, T_a)
    gas_g.gas[y_init.>radius,1,m] = gas_init

end

#R = 
#
#ambient = AmbientConditions(u_a, T_a, 1.0, 1.0, R, p, T0, u0)
#
##Solve for T and u at all steps: NO CHEMISTRY
#x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)
#x_save = x
#y_save = y
#T_save[m, :, :] = T
#u_save[m, :, :] = u

PyObject <cantera.composite.Solution object at 0x7f848b9bd9e0>

UndefKeywordError: UndefKeywordError: keyword argument dims not assigned

In [4]:
# Computing u and T independent of chemistry
x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)

UndefVarError: UndefVarError: solve_exhaust_flow not defined